In [1]:
import numpy as np
import pandas as pd

In [2]:
# Dataset is from Kaggle's Depression and Academic performance of students
# https://www.kaggle.com/datasets/kanerudolph/depression-and-academic-performance-of-students
path = "Effects of Depression  Anxiety on Academic Performance among the students (respuestas).xlsx"
df = pd.read_excel(path)

In [3]:
# Insert missing values
np.random.seed(1234)
for col in df.columns:
    df.loc[df.sample(frac=0.1).index, col] = np.nan

## Crosstabulation 

In [4]:
sex = "Gender:"
age = "Age:"
sex_age = "sex_age"

In [5]:
tab = pd.crosstab(
        df[sex].replace(np.nan,"missing"), 
        df[age].replace(np.nan,"missing"), 
        margins=True)
tab_per = ( 
    pd.crosstab(
        df[sex].replace(np.nan,"missing"), 
        df[age].replace(np.nan,"missing"), 
        margins=True, 
        normalize="index"
    ).mul(100)
     .round(2)
)

In [6]:
display(tab)
display(tab_per)

Age:,18 years or less,19 to 24 years,25 years and above,missing,All
Gender:,,,,,
Female,26,125,101,31,283
Male,2,14,16,2,34
missing,1,19,13,2,35
All,29,158,130,35,352


Age:,18 years or less,19 to 24 years,25 years and above,missing
Gender:,,,,
Female,9.19,44.17,35.69,10.95
Male,5.88,41.18,47.06,5.88
missing,2.86,54.29,37.14,5.71
All,8.24,44.89,36.93,9.94


## Continuous to category 

In [7]:
gpa = 'Your Last Semester GPA: '
df[gpa] = pd.to_numeric(df[gpa], errors='coerce')
df[gpa].describe()

count    316.000000
mean       2.988703
std        0.760973
min        1.000000
25%        2.300000
50%        3.350000
75%        3.350000
max        4.000000
Name: Your Last Semester GPA: , dtype: float64

In [8]:
gpa_cate = "gpa_cate"
labels = [">=1,<2", ">=2,<3",">=3,<=4"]
df[gpa_cate] = pd.cut(df[gpa], bins=[1,2,3,4.1], right=False, labels=labels)
df[gpa_cate].value_counts()

>=3,<=4    181
>=2,<3     114
>=1,<2      21
Name: gpa_cate, dtype: int64

In [9]:
# Way 1
df[gpa_cate] = (df[gpa]
                .mask(df[gpa].isnull(),"missing")
                .mask(df[gpa] <= 4, labels[2])
                .mask(df[gpa] < 3, labels[1])
                .mask(df[gpa] < 2, labels[0])
               )
# Way 2
df[gpa_cate] = (df[gpa]
                .mask(df[gpa].isnull(),"missing")
                .mask(df[gpa].le(4), labels[2])
                .mask(df[gpa].lt(3), labels[1])
                .mask(df[gpa].lt(2), labels[0])
               )
df[gpa_cate].value_counts()

>=3,<=4    181
>=2,<3     114
missing     36
>=1,<2      21
Name: gpa_cate, dtype: int64

#  Scales to category

In [10]:
qs = {f"q{i}" : df.columns[i+2] for i in range(1,7)}
qs = {**qs, **{i:f"q{i}_cate" for i in range(1,7) }}

In [11]:
# Way 1
for i in range(1,7):
    c = qs[f"q{i}"]
    df[qs[i]] = df[c].replace({1:0,2:0,3:1,4:1})
    
# Way 2
for i in range(1,7):
    c = qs[f"q{i}"]
    df[qs[i]] = (df[c]
                 .mask(df[c].apply(lambda x: x in [1,2]), 0)
                 .mask(df[c].apply(lambda x: x in [3,4]), 1)
                )
# Way 3
for i in range(1,7):
    c = qs[f"q{i}"]
    df[qs[i]] = (df[c]
                 .mask(df[c].eq(1) | df[c].eq(2), 0)
                 .mask((df[c] == 3) | (df[c] == 4) , 1)
                )

### Calculate Score 

In [12]:
cols = [f"q{i}_cate" for i in range(1,7)]

In [13]:
sum1 = df[cols].sum(axis=1, skipna=False)
sum2 = df[cols].replace(np.nan,1).sum(axis=1, skipna=False)
sum3 = df[cols].replace(np.nan,0).sum(axis=1, skipna=False)

scale = (sum1
        .mask(sum1.isnull(), "missing")
        .mask( (sum1 >=4 ) | (sum3 >= 4), ">=4")
        .mask( sum1 <= 3, "2-3")
        .mask( (sum1 <= 1) | (sum2 <= 1), "<=1")
        )

In [14]:
sum1.value_counts(dropna=False)

NaN    169
3.0     35
2.0     28
0.0     27
5.0     25
4.0     24
1.0     22
6.0     22
dtype: int64

In [15]:
scale.value_counts()

>=4        128
missing     85
<=1         76
2-3         63
dtype: int64